## Delay Theorem (2)
An objective to minimize the cost for delay can be considered to induce a complete order upon two separation-identical aircraft $i$ and $j$ where
$b_i \leq b_j$ and $r_i \leq r_j$.



In [26]:
from z3 import *
s = Solver()


Then, we define the following:
- $\delta \in \mathbb{Z} \to \mathbb{Z} \to \mathbb{R}$ (separation-matrix)
- $r_i \in \mathbb{Z} \to \mathbb{R}$ (release time)
- $b_i \in \mathbb{Z} \to \mathbb{R}$ (base time)

then
1. Non-negativity of release times / $r_x$:
$$ r_x \geq 0 \quad \forall x \in S $$

2. Non-negativity of base times / $b_i$
$$ b_x \geq 0 \quad \forall x \in S $$

3. Non-negativity of $\delta$
$$ \delta(x, y) \geq 0 \quad \forall x \in (S \times S) $$

4. Identical separations between `a` and `b` $\delta_a = \delta_b$
$$ \delta(a, x) = \delta(b, x) \wedge \delta(x, a) = \delta(x, b) \quad \forall x \in S $$


In [27]:

δ = Function('δ', IntSort(), IntSort(), RealSort())
R = Function('R', IntSort(), RealSort())
B = Function('B', IntSort(), RealSort())

# --- aircraft etc as before ---
p1, a, p2, b, p3 = Ints('p1 a p2 b p3')
ac = [p1, a, p2, b, p3]

x, y = Ints('x y')
s.add(ForAll([x], R(x) >= 0))
s.add(ForAll([x], B(x) >= 0))
s.add(ForAll([x], R(x) >= B(x)))
s.add(ForAll([x, y], δ(x, y) >= 0))

# δ-identical a/b
for x_ in ac:
    s.add(δ(a, x_) == δ(b, x_))
    s.add(δ(x_, a) == δ(x_, b))



In the original objective function, costs are raised to an equity variable $\alpha$, which is conflicts with the decidability of SMT logic. Instead, we pose $\alpha$ as an uninterpreted, strictly monotonic function, filling the same role as the original exponentiation but in an SMT decidable way.

$$ x \leq y \implies \alpha (x) < \alpha (y) \quad \forall x,\ y \in \mathbb{R} $$
$$ x \geq 0 \implies \alpha (x) \geq 0 \quad \forall x \in \mathbb{R} $$
$$ α(0) == 1 $$


In [28]:

α = RealVal(2);


Then define some helper functions to encode a $max$ function symbolically. Define a recursive function to compute a symbolic takeoff time for a given sequence.

In [29]:
# zmax and T computation
from itertools import product
from functools import reduce

def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs): return reduce(lambda a, b: zmax(a, b), xs)

def compute_T(seq):
    T = {seq[0]: R(seq[0])}
    for i in range(1, len(seq)):
        preds = seq[:i]
        T[seq[i]] = zmax(R(seq[i]), zmax_list([T[x] + δ(x, seq[i]) for x in preds]))
    return T


Define sequences where $a$ preceeds $b$ and $b$ preceeds $a$

In [30]:

S1, S2 = [p1, a, p2, b, p3], [p1, b, p2, a, p3]
T1, T2 = compute_T(S1), compute_T(S2)

W1 = Real('W1')
s.add(W1 > 0)
def delay_cost(T):
    return Sum(([W1 * ((T[x] - B(x) ** α)) for x in ac]))

D1, D2 = delay_cost(T1), delay_cost(T2)
s.add(R(a) <= B(b) / R(a))
s.add(D1 > D2)


In [31]:

# Check With Z3

print(s.check())
if s.check() == sat:
    print("\nCounterexample found:\n")
    m = s.model()
    for d in m.decls():
        print(f"{d.name()} = {m[d]}")
elif s.check() == unsat:
    print("\nUNSAT — theorem holds (no counterexample found)\n")
else:
    print("Unkown")

sat

Counterexample found:

a = 2
p1 = 3
p2 = 5
p3 = 6
b = 4
W1 = 1
δ = [(3, 2) -> 3/8,
 (3, 4) -> 3/8,
 (3, 6) -> 2,
 (5, 6) -> 25/16,
 else -> 0]
/0 = [else -> 5/8]
B!48 = [else ->
 If(And(Not(Var(0) == 2), Not(Var(0) == 4), Var(0) == 6),
    -65/32,
    If(Or(And(Not(Var(0) == 2), Var(0) == 4),
          And(Not(Var(0) == 2),
              Not(Var(0) == 4),
              Not(Var(0) == 6),
              Not(Var(0) == 5)),
          And(Not(Var(0) == 2),
              Not(Var(0) == 4),
              Not(Var(0) == 6),
              Var(0) == 5)),
       0,
       If(Var(0) == 2, -1/2, 49)))]
R = [else ->
 -1*B!48(Var(0)) +
 If(Or(And(Not(Var(0) == 2),
           Not(Var(0) == 4),
           Not(Var(0) == 6),
           Not(Var(0) == 5)),
       And(Not(Var(0) == 2), Not(Var(0) == 4), Var(0) == 6),
       And(Not(Var(0) == 2),
           Not(Var(0) == 4),
           Not(Var(0) == 6),
           Var(0) == 5),
       Var(0) == 2),
    0,
    If(And(Not(Var(0) == 2), Var(0) == 4), 5/16, 50